In [1]:
f = 'Alameda County Parcel Boundaries.geojson'

Format info for a .geojson file found here: http://geojson.org/geojson-spec.html

This file has a small number of line breaks and is 500 MB in size: while it might be able to read into memory all at once, I'd like to make this analysis memory efficient in case a larger file comes along.  So, read first thousand characters to see what the format looks like.

In [2]:
fid = open(f, 'r')
data = []
for x in range(700) :
    data.append(fid.read(1))
  
fid.close()

data = ''.join(data)
print(data)

{
  "type": "FeatureCollection",
  "features": [
    {"type":"Feature","properties":{"shape_st_2":"682.98789617","fid_parcel":"1","shape_stle":"682.98789617","book":"48","page":"6298","clca_categ":null,"date_updat":"2004-08-09","centroid_y":"2101850.09674882","editor":"Public Works","sub_parcel":null,"shape_st_4":"682.987896169","apn_sort":"048 629806800","shape_st_1":"27216.84179688","parcel":"68","centroid_x":"6093120.57110886","date_creat":"2004-08-09","shape_star":"27216.84179688","apn":"48-6298-68","shape_st_3":"27216.8417969","comments":null},"geometry":{"type":"MultiPolygon","coordinates":[[[[-122.12036188789048,37.756209609069614],[-122.12078602691324,37.75583754643068],[-122.1207869


Use ijson to parse this further...

In [3]:
import ijson
with open(f, 'r') as fid :
    for (i,o) in zip(range(10), ijson.parse(fid)) :
        print(o)
    fid.close()

('', 'start_map', None)
('', 'map_key', 'type')
('type', 'string', 'FeatureCollection')
('', 'map_key', 'features')
('features', 'start_array', None)
('features.item', 'start_map', None)
('features.item', 'map_key', 'type')
('features.item.type', 'string', 'Feature')
('features.item', 'map_key', 'properties')
('features.item.properties', 'start_map', None)


Here's the structure of the second item (first item is too long).

In [4]:
import pprint

with open(f, 'r') as fid :
    objects = ijson.items(fid, 'features.item')
    for (i, o) in zip(range(2), objects) :
        pass
    fid.close()
pprint.pprint(o)

{'geometry': {'coordinates': [[[[Decimal('-122.12383758245336'),
                                 Decimal('37.75571233182055')],
                                [Decimal('-122.12417229112891'),
                                 Decimal('37.755581544780114')],
                                [Decimal('-122.12417786881252'),
                                 Decimal('37.755594075381474')],
                                [Decimal('-122.12424662138503'),
                                 Decimal('37.75574856188482')],
                                [Decimal('-122.12388534031416'),
                                 Decimal('37.75586057852918')],
                                [Decimal('-122.1238793355483'),
                                 Decimal('37.75583734460038')],
                                [Decimal('-122.12387281699273'),
                                 Decimal('37.75581420454203')],
                                [Decimal('-122.12386579509725'),
                               

How many records are there?

In [ ]:
objects = ijson.items(fid, 'features.item.properties.fid_parcel')
for (i, o) in enumerate(objects):
    pass
print(i+1)

The output would be 399212.  Since it takes about a second to parse 1000 records, that took about 7 minutes to compute.

Now, let's extract all addresses within 0.5 km, and save it as a Panda HDF5 object.

Is there a chance that some records have a street address?  No: run 'grep -l "addr" <FILENAME>', and no dice.  OK, so how to convert this to valid addresses?

Want latitude = 37.805553, longitude = -122.239429, which corresponds to the Armenian church.